In [1]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import json

In [4]:
client = Elasticsearch()

In [2]:
!ls

bluecross-al-drugs.json  elasticsearch-demo.ipynb provider.json
drug.json                plan.json


In [288]:
plan = json.load(open('plan.json'))   # (open("bluecross-al-drugs.json"))

In [278]:
drug = json.load(open('drug.json'))

In [165]:
mappings = json.load(open('mappings.json'))

In [174]:
with open('encode2.json', 'w') as outfile:
    json.dump(mappings['mappings'], outfile)

In [171]:
# mappings['mappings']

def addEncode(fields):    
    if ('type' in fields) and ('type' not in fields['type']): #(len(fields)==1):
        fields['encode'] = 0
    else:
        for v in fields.values():
            addEncode(v)
        
# addEncode(mappings)

# mappings['mappings']

###Load encode definition

In [297]:
# read file
encode = json.load(open('encode2.json'))
field_list = {}

def getEncodeFields(encode_def, encode_var, path=''):
    if 'encode' in encode_def and encode_def['encode'] == 1:
        encode_var[path[1:]] = encode_def['type']
    elif 'encode' not in encode_def:
        for f in encode_def:
            getEncodeFields(encode_def[f], encode_var, path + '/' + f)
        
getEncodeFields(encode, field_list)

# field_list.keys()[0].split('/')#[1:]
field_list

{u'drug/properties/plans/properties/drug_tier': u'string',
 u'drug/properties/plans/properties/quantity_limit': u'boolean',
 u'plan/properties/formulary/properties/cost_sharing/properties/coinsurance_opt': u'string',
 u'plan/properties/formulary/properties/cost_sharing/properties/coinsurance_rate': u'float',
 u'plan/properties/formulary/properties/cost_sharing/properties/copay_amount': u'float',
 u'plan/properties/formulary/properties/cost_sharing/properties/copay_opt': u'string',
 u'plan/properties/formulary/properties/cost_sharing/properties/pharmacy_type': u'string',
 u'plan/properties/formulary/properties/drug_tier': u'string',
 u'plan/properties/formulary/properties/mail_order': u'boolean',
 u'plan/properties/network/properties/network_tier': u'string',
 u'plan/properties/plan_id_type': u'string'}

###Retrieve unique values

In [299]:
drug_plan_data={'plan':plan,'drug':drug}

In [331]:
# get unique value for each string encode field (prepare for on-hot-encode)

from sets import Set

def getFieldValue(data, key, val, add_call):
    if len(key) == 1:        
        add_call(val, data[key[0]])
    elif type(data) is list: # key[0] == 'properties':
        for d in data:
            getFieldValue(d, key[1:], val, add_call)
    else:
        getFieldValue(data[key[0]], key[1:], val, add_call)
        
def add_set(vSet, val):
    vSet.add(val)
    
def add_list(vList, val):
    vList.append(val)
        
        
# 
uni_val = {}
for k,v in field_list.items():
    if True: # v=='string':
        uni = Set()
        getFieldValue(drug_plan_data, k.split('/'), uni, add_set)
        uni_val[k] = list(uni)

# uni_val

###Investigate cost sharing policy for each drug tier

In [277]:
len(bluecross_drugs_list)
for p in bluecross_drugs_list:
#     print [f['drug_tier'] for f in p['formulary']]
    for f in p['formulary']:
        if f['drug_tier']=='VALUE':
            print [c['pharmacy_type']+'/'+str(c['coinsurance_opt'])+'/'+c['copay_opt'] for c in f['cost_sharing']] 
#     print '\n'


[u'3-MONTH-IN-MAIL/None/BEFORE-DEDUCTIBLE', u'3-MONTH-IN-RETAIL/None/BEFORE-DEDUCTIBLE', u'3-MONTH-OUT-RETAIL/None/BEFORE-DEDUCTIBLE']
[u'3-MONTH-IN-MAIL/None/BEFORE-DEDUCTIBLE', u'3-MONTH-IN-RETAIL/None/BEFORE-DEDUCTIBLE', u'3-MONTH-OUT-RETAIL/None/BEFORE-DEDUCTIBLE']
[u'3-MONTH-IN-MAIL/None/BEFORE-DEDUCTIBLE', u'3-MONTH-IN-RETAIL/None/BEFORE-DEDUCTIBLE', u'3-MONTH-OUT-RETAIL/None/BEFORE-DEDUCTIBLE']
[u'3-MONTH-IN-MAIL/None/BEFORE-DEDUCTIBLE', u'3-MONTH-IN-RETAIL/None/BEFORE-DEDUCTIBLE', u'3-MONTH-OUT-RETAIL/None/BEFORE-DEDUCTIBLE']
[u'3-MONTH-IN-MAIL/None/BEFORE-DEDUCTIBLE', u'3-MONTH-IN-RETAIL/None/BEFORE-DEDUCTIBLE', u'3-MONTH-OUT-RETAIL/None/BEFORE-DEDUCTIBLE']
[u'3-MONTH-IN-MAIL/None/NO-CHARGE', u'3-MONTH-IN-RETAIL/None/BEFORE-DEDUCTIBLE', u'3-MONTH-OUT-RETAIL/None/BEFORE-DEDUCTIBLE']
[u'3-MONTH-IN-MAIL/None/NO-CHARGE', u'3-MONTH-IN-RETAIL/None/BEFORE-DEDUCTIBLE', u'3-MONTH-OUT-RETAIL/None/BEFORE-DEDUCTIBLE']
[u'3-MONTH-IN-MAIL/None/BEFORE-DEDUCTIBLE', u'3-MONTH-IN-RETAIL/None/BE

###Generate feature vector with integer value

In [337]:
k=field_list.keys()[0].split('/')[2:]
test = []
getFieldValue(drug_plan_data['plan'][0], k, test, add_list)

print [uni_val.values()[0].index(x) for x in test]

[0, 1, 3, 0, 1, 3, 0, 1, 3, 2, 1, 0, 1, 3, 1]


In [314]:
uni_val.values()[0].index('1-MONTH-IN-RETAIL')

2

In [510]:
import numpy as np

features = []

for k,v in uni_val.items():
    if not k.startswith('plan'):
        continue
    print k
    fea = []
    for pl in drug_plan_data['plan']:    
        if len(pl['formulary'])==0:
            continue
        plan_val = []
        getFieldValue(pl, k.split('/')[2:], plan_val, add_list)             
        fea.append([v.index(x) for x in plan_val] if field_list[k] == 'string' else plan_val)
    # assemble
    features.append(np.array(fea))            
        
            

plan/properties/formulary/properties/cost_sharing/properties/pharmacy_type
plan/properties/formulary/properties/drug_tier
plan/properties/network/properties/network_tier
plan/properties/formulary/properties/cost_sharing/properties/coinsurance_rate
plan/properties/plan_id_type
plan/properties/formulary/properties/mail_order
plan/properties/formulary/properties/cost_sharing/properties/coinsurance_opt
plan/properties/formulary/properties/cost_sharing/properties/copay_opt
plan/properties/formulary/properties/cost_sharing/properties/copay_amount


In [528]:
features[1]

array([[6, 4, 0, 1, 5, 2, 3],
       [6, 4, 0, 1, 5, 2, 3],
       [6, 4, 0, 1, 5, 2, 3],
       [6, 4, 0, 1, 5, 2, 3],
       [6, 4, 0, 1, 5, 2, 3],
       [6, 4, 0, 1, 5, 2, 3],
       [6, 4, 0, 1, 5, 2, 3],
       [6, 4, 0, 1, 5, 2, 3],
       [6, 4, 0, 1, 5, 2, 3],
       [6, 4, 0, 1, 5, 2, 3],
       [6, 4, 0, 1, 5, 2, 3]])

In [482]:
features = []

for pl in drug_plan_data['plan']:
    p_fea = []
    for k,v in uni_val.items():
        if not k.startswith('plan'):
            continue
        fea_val = []
        getFieldValue(pl, k.split('/')[2:], fea_val, add_list)
        p_fea += [v.index(x) for x in fea_val] if field_list[k]=='string' else fea_val
    features.append(np.array(p_fea))
        

In [495]:
[len(x) for x in features]

[91, 91, 91, 91, 91, 91, 91, 91, 91, 91, 91, 2, 2, 2, 2]

In [506]:
drug_plan_data['drug'][1100]['plans']

[{u'drug_tier': u'PREFERRED',
  u'plan_id': u'73836AK0750002',
  u'plan_id_type': u'HIOS-PLAN-ID',
  u'prior_authorization': False,
  u'quantity_limit': True,
  u'step_therapy': True},
 {u'drug_tier': u'PREFERRED',
  u'plan_id': u'73836AK0750001',
  u'plan_id_type': u'HIOS-PLAN-ID',
  u'prior_authorization': False,
  u'quantity_limit': True,
  u'step_therapy': True},
 {u'drug_tier': u'PREFERRED',
  u'plan_id': u'73836AK0750003',
  u'plan_id_type': u'HIOS-PLAN-ID',
  u'prior_authorization': False,
  u'quantity_limit': True,
  u'step_therapy': True},
 {u'drug_tier': u'PREFERRED',
  u'plan_id': u'73836AK0750004',
  u'plan_id_type': u'HIOS-PLAN-ID',
  u'prior_authorization': False,
  u'quantity_limit': True,
  u'step_therapy': True},
 {u'drug_tier': u'PREFERRED',
  u'plan_id': u'73836AK0840001',
  u'plan_id_type': u'HIOS-PLAN-ID',
  u'prior_authorization': False,
  u'quantity_limit': True,
  u'step_therapy': True},
 {u'drug_tier': u'PREFERRED',
  u'plan_id': u'73836AK0850001',
  u'plan_id_

In [507]:
[len(d['plans']) for d in drug_plan_data['drug']].index(22)

266

In [508]:
drug_plan_data['drug'][266]['plans']

[{u'drug_tier': u'PREFERRED',
  u'plan_id': u'73836AK0750001',
  u'plan_id_type': u'HIOS-PLAN-ID',
  u'prior_authorization': False,
  u'quantity_limit': False,
  u'step_therapy': False},
 {u'drug_tier': u'BRAND',
  u'plan_id': u'73836AK0750001',
  u'plan_id_type': u'HIOS-PLAN-ID',
  u'prior_authorization': False,
  u'quantity_limit': False,
  u'step_therapy': False},
 {u'drug_tier': u'PREFERRED',
  u'plan_id': u'73836AK0750003',
  u'plan_id_type': u'HIOS-PLAN-ID',
  u'prior_authorization': False,
  u'quantity_limit': False,
  u'step_therapy': False},
 {u'drug_tier': u'BRAND',
  u'plan_id': u'73836AK0750003',
  u'plan_id_type': u'HIOS-PLAN-ID',
  u'prior_authorization': False,
  u'quantity_limit': False,
  u'step_therapy': False},
 {u'drug_tier': u'PREFERRED',
  u'plan_id': u'73836AK0750004',
  u'plan_id_type': u'HIOS-PLAN-ID',
  u'prior_authorization': False,
  u'quantity_limit': False,
  u'step_therapy': False},
 {u'drug_tier': u'BRAND',
  u'plan_id': u'73836AK0750004',
  u'plan_id_ty

In [8]:
def process_formulary_url(input_list, es):
    # process each drug in the formulary
    for drug in input_list:
        # check optional fields and add them if they are missing
        for plan in drug['plans']:
            if 'prior_authorization' not in plan:
                plan['prior_authorization'] = 'false'
            if 'step_therapy' not in plan:
                plan['step_therapy'] = 'false'
            if 'quantity_limit' not in plan:
                plan['quanitity_limit'] = 'false'
        es.index(index='data', doc_type='drug', body=drug)

In [10]:
process_formulary_url(bluecross_drugs_list, client)

In [22]:
def query(query_text, client):
    s = Search().using(client).query("match", drug_name=query_text)
    response = s.execute()
    for hit in response:
        print('/%s/%s/%s returned with score %f' % (hit.meta.index, hit.meta.doc_type, hit.meta.id, hit.meta.score))        
        print "DRUG NAME: {}".format(hit.drug_name)
        print "PLANS: "
        print hit.plans
        print "---------------------\n"

In [34]:
query("VIAGRA", client)

/data/drug/AVVpD5xaHBTA23S7lkyA returned with score 4.048774
DRUG NAME: VIAGRA       TAB 100MG
PLANS: 
[{u'quantity_limit': True, u'plan_id': u'46944AL0460001', u'plan_id_type': u'HIOS-PLAN-ID', u'step_therapy': False, u'prior_authorization': True, u'drug_tier': u'TIER-THREE'}, {u'quantity_limit': True, u'plan_id': u'46944AL0560001', u'plan_id_type': u'HIOS-PLAN-ID', u'step_therapy': False, u'prior_authorization': True, u'drug_tier': u'TIER-THREE'}, {u'quantity_limit': True, u'plan_id': u'46944AL0410001', u'plan_id_type': u'HIOS-PLAN-ID', u'step_therapy': False, u'prior_authorization': True, u'drug_tier': u'TIER-THREE'}, {u'quantity_limit': True, u'plan_id': u'46944AL0420001', u'plan_id_type': u'HIOS-PLAN-ID', u'step_therapy': False, u'prior_authorization': True, u'drug_tier': u'TIER-THREE'}, {u'quantity_limit': True, u'plan_id': u'46944AL0470001', u'plan_id_type': u'HIOS-PLAN-ID', u'step_therapy': False, u'prior_authorization': True, u'drug_tier': u'TIER-THREE'}, {u'quantity_limit': T

In [13]:
query("MORPHINE", client)

/data/drug/AVVpD8vhHBTA23S7lmq2 returned with score 3.353347
DRUG NAME: MORPHINE SUL TAB 15MG
PLANS: 
[{u'quantity_limit': True, u'plan_id': u'46944AL0370001', u'plan_id_type': u'HIOS-PLAN-ID', u'step_therapy': False, u'prior_authorization': False, u'drug_tier': u'TIER-TWO'}, {u'quantity_limit': True, u'plan_id': u'46944AL0280001', u'plan_id_type': u'HIOS-PLAN-ID', u'step_therapy': False, u'prior_authorization': False, u'drug_tier': u'TIER-TWO'}, {u'quantity_limit': True, u'plan_id': u'46944AL0340001', u'plan_id_type': u'HIOS-PLAN-ID', u'step_therapy': False, u'prior_authorization': False, u'drug_tier': u'TIER-TWO'}, {u'quantity_limit': True, u'plan_id': u'46944AL0310001', u'plan_id_type': u'HIOS-PLAN-ID', u'step_therapy': False, u'prior_authorization': False, u'drug_tier': u'TIER-TWO'}]
---------------------

/data/drug/AVVpD8vjHBTA23S7lmq3 returned with score 3.353347
DRUG NAME: MORPHINE SUL TAB 30MG
PLANS: 
[{u'quantity_limit': True, u'plan_id': u'46944AL0370001', u'plan_id_type': u

In [452]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit([[0, 0, 3.14], [1, 1, 0.15], [0, 2, 1.15], [1, 0, 2.0]])  

OneHotEncoder(categorical_features='all', dtype=<type 'float'>,
       n_values='auto', sparse=True)

In [44]:
enc.fit([[10, 0, 3], [1, 1, 0], [10, 2, 1], [1, 0, 2]])  

OneHotEncoder(categorical_features='all', dtype=<type 'float'>,
       n_values='auto', sparse=True)

In [453]:
enc.transform([[1, 2, 3.15]]).toarray()

array([[ 0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  1.]])

In [61]:
ord('h')

104